In [20]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA



In [18]:
## Read the ppdfs from the folder
loader=PyPDFDirectoryLoader("./us_census")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[0]

IndexError: list index out of range

In [21]:
import pandas as pd

# Load the CSV file
data_file = "hadith_data.csv"
df = pd.read_csv(data_file)

# Extract the "Hadith Text" column
hadith_texts = df["Hadith Text"].dropna().tolist()


In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Split each Hadith text into chunks
final_documents = []
for text in hadith_texts:
    chunks = text_splitter.split_text(text)
    final_documents.extend(chunks)

# Each chunk is a part of the Hadith text
print(final_documents[:5])  # Display the first 5 chunks


['Allah\'s Messenger (ﷺ) said, "While I was at Mecca the roof of my house was opened and Gabriel descended, \nopened my chest, and washed it with Zamzam water. Then he brought a golden tray full of wisdom \nand faith and having poured its contents into my chest, he closed it. Then he took my hand and \nascended with me to the nearest heaven, when I reached the nearest heaven, Gabriel said to the \ngatekeeper of the heaven, \'Open (the gate).\' The gatekeeper asked, \'Who is it?\' Gabriel answered: \n\'Gabriel.\' He asked, \'Is there anyone with you?\' Gabriel replied, \'Yes, Muhammad I is with me.\' He \nasked, \'Has he been called?\' Gabriel said, \'Yes.\' So the gate was opened and we went over the nearest \nheaven and there we saw a man sitting with some people on his right and some on his left. When he \nlooked towards his right, he laughed and when he looked toward his left he wept. Then he said, \n\'Welcome! O pious Prophet and pious son.\' I asked Gabriel, \'Who is he?\' He repl

In [23]:
## Embedding Using Huggingface
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

# Example texts
texts = ["While I was at Mecca the roof of my house was opened...",
         "Gabriel said to the gatekeeper of the heaven, 'Open (the gate).'"]

# Generate embeddings
embeddings = huggingface_embeddings.embed_documents(texts)

# Display the embeddings
print(embeddings[0])  # Prints the embedding vector for the first text



[0.019624490290880203, 0.04553646966814995, 0.03260500729084015, -0.0031482118647545576, 0.022959046065807343, -0.006676042918115854, 0.08351331204175949, 0.001752863870933652, 0.04383356496691704, -0.0391056090593338, 0.0016460524639114738, 0.019809318706393242, 0.010862298309803009, 0.003174691693857312, 0.06661178171634674, 0.011614515446126461, -0.057227615267038345, 0.03592041879892349, -0.09562207013368607, 0.04215120151638985, -0.09020876884460449, -0.046471692621707916, -0.008220091462135315, -0.056422024965286255, -0.02743370085954666, 0.10401047766208649, 0.015995267778635025, 0.030536523088812828, -0.013632283546030521, -0.06252141296863556, -0.0202234648168087, -0.0506729818880558, 0.03919683024287224, -0.01572345569729805, 0.03516285493969917, 0.05221071094274521, -0.053536150604486465, -0.03585733473300934, 0.03800337761640549, 0.01365664042532444, 0.08970610797405243, 0.03337949514389038, 0.004542062059044838, -0.0356660820543766, 0.006378337275236845, -0.032881565392017

In [24]:
from langchain.vectorstores import FAISS

# Create a FAISS vector store
vector_store = FAISS.from_texts(texts, huggingface_embeddings)

# Save the FAISS index for future use
vector_store.save_local("faiss_store")


In [25]:
import faiss
import numpy as np

# Example texts
texts = [
    "While I was at Mecca the roof of my house was opened...",
    "Gabriel said to the gatekeeper of the heaven, 'Open (the gate).'"
]

# Generate embeddings
embeddings = huggingface_embeddings.embed_documents(texts)

# Convert embeddings to a NumPy array with dtype float32
embeddings_array = np.array(embeddings).astype('float32')  # FAISS requires float32


In [39]:
from langchain.vectorstores import FAISS


# Create a FAISS vector store
vector_store = FAISS.from_texts(texts, huggingface_embeddings)

# Save the FAISS store for reuse
vector_store.save_local("faiss_store")

# Load the FAISS store
loaded_store = FAISS.load_local("faiss_store", huggingface_embeddings)


ValueError: The de-serialization relies loading a pickle file. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine.You will need to set `allow_dangerous_deserialization` to `True` to enable deserialization. If you do this, make sure that you trust the source of the data. For example, if you are loading a file that you created, and know that no one else has modified the file, then this is safe to do. Do not set this to `True` if you are loading a file from an untrusted source (e.g., some random site on the internet.).

In [38]:
from langchain.vectorstores import FAISS

# Save the FAISS vector store
vector_store.save_local("faiss_store")

# Load the FAISS store with explicit permission for deserialization
loaded_store = FAISS.load_local(
    "faiss_store",
    huggingface_embeddings,
    allow_dangerous_deserialization=True
)


In [37]:
# Ensure embeddings are a numpy array
embeddings_array = np.array(embeddings)

# Create a FAISS vector store
vector_store = FAISS.from_documents(
    texts=texts,
    embeddings=embeddings_array,
    embedding=huggingface_embeddings,
)

# Save the FAISS store for reuse
vector_store.save_local("faiss_store")

TypeError: VectorStore.from_documents() missing 1 required positional argument: 'documents'

In [13]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

AttributeError: 'str' object has no attribute 'page_content'

In [11]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [14]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [15]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000159F4860C10> search_kwargs={'k': 3}


In [16]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']=""

The Hugging Face Hub is an platform with over 350k models, 75k datasets, and 150k demo apps (Spaces), all open source and publicly available, in an online platform where people can easily collaborate and build ML together.

In [17]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}

)
query="What is the health insurance coverage?"
hf.invoke(query)

e:\Langchain\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


'What is the health insurance coverage?\n\nThe health insurance coverage is a contract between the insurer and the insured. The insurer agrees to pay the insured for the medical expenses incurred by the insured. The insured agrees to pay the premium to the insurer.\n\nWhat is the health insurance coverage?\n\nThe health insurance coverage is a contract between the insurer and the insured. The insurer agrees to pay the insured for the medical expenses incurred by the insured.'

In [ ]:
#Hugging Face models can be run locally through the HuggingFacePipeline class.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf 
llm.invoke(query)

In [22]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [24]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [25]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [28]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [29]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])


Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

comparison of ACS and CPS ASEC measures 
of health insurance coverage, refer to < www.
census.gov/topics/health/health-insurance/
guidance.html >.
9 Respondents may have more than one 
health insurance coverage type at the time 
of interview. As a result, adding the total 
number of people with private coverage and 
the total number with public coverage will 
sum to more than the total number with any 
coverage.• From 2021 to 2022, nine states 
reported increases in private 
coverage, while seven reported 
decreases (Appendix Table B-2). 
DIFFERENCES IN THE 
UNINSURED RATE BY STATE 
IN 2022
In 2022, uninsured rates at the 
time of interview ranged across 
states from a low of 2.4 percent 
in Massachusetts to a high of 16.6 
percent in Texas, compared to the 
national rate of 8.0 percent.10 Ten 
of the 15 states with uninsured 
10 The uninsured rates in the Distr